In [14]:
import requests
from bs4 import BeautifulSoup

In [8]:
r = requests.get('http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247')

In [9]:
r.status_code

200

In [10]:
r.headers['content-type']

'text/html; charset=ISO-8859-15'

In [11]:
r.encoding

'ISO-8859-15'

In [12]:
r.text

u'<html>\n   <head>\n      <META http-equiv="Content-Type" content="text/html; charset=iso-8859-1">\n      <title>IS-Academia</title>\n      <link rel="stylesheet" type="text/css" href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style">\n      <link rel="stylesheet" type="text/css" href="!ewxslsource.css?ww_x_path=Eqnx.Ajax.PageNL_css"><script type="text/javascript" src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.core"></script><script type="text/javascript" src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.drag"></script><script type="text/javascript" src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.event"></script><script type="text/javascript" src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.dom"></script><script type="text/javascript" src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.ewnet.namespace"></script><script type="text/javascript" src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.ewnet.core"></script><script type="text/javascript" src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.ewne

In [13]:
r.json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Good Resource for Beautiful soup: https://www.crummy.com/software/BeautifulSoup/bs4/doc/

In [16]:
soup = BeautifulSoup(r.text, 'html.parser')
print(soup.prettify())

<html>
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type">
   <title>
    IS-Academia
   </title>
   <link href="gedpublicreports.css?ww_x_path=Gestac.Moniteur.Style" rel="stylesheet" type="text/css">
    <link href="!ewxslsource.css?ww_x_path=Eqnx.Ajax.PageNL_css" rel="stylesheet" type="text/css">
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.core" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.drag" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.event" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.X.dom" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.ewnet.namespace" type="text/javascript">
     </script>
     <script src="gedpublicreports.txt?ww_x_path=Eqnx.Ajax.ewnet.core" type="text/javascript">
     </script>
     <script src="gedpubl

In [17]:
soup.title

<title>IS-Academia</title>

In [19]:
soup.title

soup.title.name


soup.title.string


soup.title.parent.name


soup.p


soup.p['class']


soup.a


soup.find_all('a')




TypeError: 'NoneType' object has no attribute '__getitem__'